In [3]:
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import sqlite3
from random import randint
from time import sleep

# Subcategories

In [ ]:
#Function to get main url Tiki
def get_url(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    return soup

In [ ]:
#Finding tag for categories 
link = get_url('https://tiki.vn/')
link.find_all('a', {'class' : 'MenuItem__MenuLink-sc-181aa19-1 fKvTQu'})

In [6]:
conn = sqlite3.connect('Tiki_main.db')
cur = conn.cursor()

In [212]:
#Creating a table named Categories
def create_table():
    query = '''
    CREATE TABLE IF NOT EXISTS categories 
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(256),
    url TEXT,
    parent_id INTEGER
    )'''
    try:
        conn.execute(query)
        conn.commit()
    except Exception as err:
        print('Error', err)

create_table()

In [ ]:
class Category():
    def __init__(self, name, url, parent_id = None, cat_id = None):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id
        
    
    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id}"
    
    def save_into_db(self):
        query = '''
        INSERT INTO categories
        (name, url, parent_id)
        VALUES (?,?,?)'''
        
        value = (self.name, self.url, self.parent_id)
        
        try:
            cur.execute(query, value)
            self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print("Error")

In [ ]:
#Function to get main categories
def main(save_db = False):
    soup = get_url('https://tiki.vn/')
    
    result = []
    
    for a in soup.find_all('a', {'class': 'MenuItem__MenuLink-sc-181aa19-1 fKvTQu'}):
        name = a.find('span', {'class': 'text'}).text
        url = a['href']
        main_cat = Category(name, url)
        
        if save_db:
            main_cat.save_into_db()
        result.append(main_cat)
        
    return result

In [ ]:
main_categories = main(save_db = True)

In [ ]:
print(len(main_categories))
#16 main categories

In [ ]:
import re

def sub_categories(parent_category, save_db = False):
    parent_url = parent_category.url
    result = []
    
    try:
        soup = get_url(parent_url)
        div_children = soup.find_all('div', {'class': 'list-group-item is-child'})
        
        for div in div_children:
            name = div.a.text.strip()
            
            name = re.sub('\s{2,}','',name)
            
            sub_url = 'https://tiki.vn/' + div.a['href']
            cat = Category(name, sub_url, parent_category.cat_id)
            
            if save_db:
                cat.save_into_db()
                
            result.append(cat)
                
    except Exception as err:
        print("Error with children categories", err)
    
    return result

In [ ]:
def get_all(categories):
    if len(categories) == 0:
        return
    
    for item in categories:
        children_categories = sub_categories(item, save_db = True)
        print(children_categories)
        get_all(children_categories)

In [ ]:
get_all(main_categories)

# Scrapping Products

In [213]:
def product_table():
    query = '''
    CREATE TABLE IF NOT EXISTS product
    (Product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    Title VARCHAR(256),
    Brand VARCHAR(256),
    Original INTEGER,
    Discounted SMALLINT,
    Final INTEGER,
    Link TEXT,
    Cat_id INTEGER)'''
    #Tom suggested I search up how to do one to many relationships in sqlite python...
    # FOREIGN KEY (Cat_id) REFERENCES categories (id)
    
    try:
        conn.execute(query)
        conn.commit()
    except Exception as err:
        print('Error', err)

In [177]:
class Product():
    def __init__(self, title, brand, original, discounted, final, link, cat_id = None, product_id = None): 
        self.title = title
        self.brand = brand
        self.original = original
        self.discounted = discounted
        self.final = final
        self.link = link 
        self.product_id = product_id 
        self.cat_id = cat_id
    
    def __repr__(self):
        return f'''
        Title: {self.title}, 
        Brand: {self.brand}, 
        Original: {self.original}, 
        Discounted: {self.discounted}, 
        Final: {self.final}, 
        Link: {self.link}'''
    
    
    def save_into_db(self):
        query = '''
        INSERT INTO product
        (Title, Brand, Original, Discounted, Final, Link, cat_id)
        VALUES (?,?,?,?,?,?,?);'''
        value = (self.title,self.brand, self.original, self.discounted, self.final, self.link, self.cat_id)
        
        try:
            cur.execute(query,value)
            self.product_id = cur.lastrowid #increment key values
            conn.commit()
            
        except Exception as err:
            print("Error", err)

## Getting all subcategories url

In [7]:
def get_all_url_id():
    sub_categories = pd.read_sql('''
    SELECT a.url, a.id
    FROM categories AS a
    LEFT JOIN categories as b ON a.id = b.parent_id
    WHERE b.id IS NULL
    ORDER BY a.name ASC''', conn)
    
    return sub_categories
#Don't use distinct, otherwise you won't get back the id value for table referencing 

In [8]:
sub_categories = get_all_url_id()

In [10]:
sub_categories

,url,id
0,https://tiki.vn//3d/c23742?src=c.1883.hamburge...,887
1,https://tiki.vn//access-point-diem-truy-cap/c4...,314
2,https://tiki.vn//access/c14884?src=c.8322.hamb...,2827
3,https://tiki.vn//action-adventure/c142?src=c.8...,2849
4,https://tiki.vn//action-adventure/c11017?src=c...,2793
...,...,...
2675,https://tiki.vn//o-cam-dien/c2021?src=c.1883.h...,557
2676,https://tiki.vn//o-khoa-vali/c8389?src=c.914.h...,2125
2677,https://tiki.vn//o-khoa/c2154?src=c.1883.hambu...,562
2678,https://tiki.vn//o-dia-quang/c28908?src=c.1846...,280


## Implementing 

In [224]:
def get_product(save_db = False):
    result = []
    try:
        #Shortcoming: I didn't take into consideration that some products don't have discount and,thus, no final price
        #But I was too deep in ... 
        
        for product in products:
            title = product['data-title']
            brand = product['data-brand']
            original = product.find('span', {'class': 'price-regular'}).text.replace('đ','').replace('.','')
            #try..except
            discounted = product.find('span', {'class': 'sale-tag sale-tag-square'}).text.replace('%','')
            #try..except 
            final = product['data-price']
            link = 'tiki.vn' + product.a['href']
            
            #See below for reference on iloc
            #The sub_categories['id'] is a np.interger, whereas our table only supports int type...
            #We need to change to int to avoid something call a BLOB type. Search google! 
            
            item = Product(title, brand, original, discounted, final, link, int(sub_categories['id'].iloc[pages]))
            if save_db:
                item.save_into_db()
            result.append(item)
    except Exception as err:
        print('F', err)
    
    return result 

In [64]:
#Testing to see if we can use .iloc to get url. See below for refrence on iloc. 
r = requests.get(sub_categories['url'].iloc[0])
soup = BeautifulSoup(r.text, 'html.parser')
product = soup.find('div', {'class':'product-item'})

In [66]:
print(len(sub_categories))

2680


In [168]:
print(type(sub_categories['id'].iloc[0]))
#Checking type for sub_categories['id']#

<class 'numpy.int64'>


In [100]:
str(sub_categories['url'].iloc[0])

'https://tiki.vn//3d/c23742?src=c.1883.hamburger_menu_fly_out_banner'

In [ ]:
sub_categories = get_all_url_id()

In [ ]:
sub_categories

In [ ]:
# Pages in range len(sub_categories) will return the link one by one 
for pages in range(0, len(sub_categories)):
    for i in range(1, 23):
        try:            
            r = requests.get(str(sub_categories['url'].iloc[pages]) + '&page=' +str(i))
            soup = BeautifulSoup(r.text, 'html.parser')
            products = soup.find_all('div', {'class':'product-item'})
        except:
            continue
        # if we can't find any products, we skip the rest of the function and move onto the next i value...
        
        if len(products) == 0:
            break
            
        result = get_product(save_db = True)
        sleep(randint(2,6))   